# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('cal_housing_clean.csv')

In [3]:
df.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
df.describe()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,28.639486,2635.763081,537.898014,1425.476744,499.539680,3.870671,206855.816909
std,12.585558,2181.615252,421.247906,1132.462122,382.329753,1.899822,115395.615874
min,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119600.000000
50%,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X = df.drop('medianHouseValue',axis=1)

In [7]:
y = df['medianHouseValue']

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.30, random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
scaler = MinMaxScaler()

In [58]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [70]:

dfXTrain = pd.DataFrame(data=scaler.transform(X_train),columns=X_train.columns,index=X_train.index)

In [71]:
dfXTest = pd.DataFrame(data=scaler.transform(X_test),columns=X_test.columns,index=X_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [72]:
df.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [64]:
import tensorflow as tf

In [73]:
housingMedianAge = tf.feature_column.numeric_column('housingMedianAge')
totalRooms = tf.feature_column.numeric_column('totalRooms')
totalBedrooms = tf.feature_column.numeric_column('totalBedrooms')
population = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
medianIncome = tf.feature_column.numeric_column('medianIncome')
#medianHouseValue = tf.feature_column.numeric_column('medianHouseValue')

In [74]:
feat_cols = [housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [79]:
input_func = tf.estimator.inputs.pandas_input_fn(x=dfXTrain,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [80]:
estimator = tf.estimator.DNNRegressor(feature_columns=feat_cols,hidden_units=[6,6,6])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_is_chief': True, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_num_worker_replicas': 1, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb66c72b320>, '_save_summary_steps': 100, '_session_config': None, '_task_id': 0, '_service': None, '_model_dir': '/tmp/tmpji1u7zxn', '_tf_random_seed': None, '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_task_type': 'worker', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [82]:
estimator.train(input_fn=input_func,steps=20000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpji1u7zxn/model.ckpt-1000
INFO:tensorflow:Saving checkpoints for 1001 into /tmp/tmpji1u7zxn/model.ckpt.
INFO:tensorflow:loss = 4.11004e+11, step = 1001
INFO:tensorflow:global_step/sec: 518.117
INFO:tensorflow:loss = 2.24783e+11, step = 1101 (0.198 sec)
INFO:tensorflow:global_step/sec: 521.752
INFO:tensorflow:loss = 5.71904e+11, step = 1201 (0.195 sec)
INFO:tensorflow:global_step/sec: 545.066
INFO:tensorflow:loss = 2.69046e+11, step = 1301 (0.181 sec)
INFO:tensorflow:global_step/sec: 573.394
INFO:tensorflow:loss = 1.00314e+11, step = 1401 (0.173 sec)
INFO:tensorflow:global_step/sec: 533.489
INFO:tensorflow:loss = 4.76876e+11, step = 1501 (0.184 sec)
INFO:tensorflow:global_step/sec: 569.219
INFO:tensorflow:loss = 2.2805e+11, step = 1601 (0.175 sec)
INFO:tensorflow:global_step/sec: 474.277
INFO:tensorflow:loss = 4.48054e+11, step = 1701 (0.211 sec)
INFO:tensorflow:global_step/sec: 616.446
INFO:ten

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [83]:
input_fn_predict = tf.estimator.inputs.pandas_input_fn(x=X_test,shuffle=False,batch_size=10,num_epochs=1)

In [90]:
pred_gen = estimator.predict(input_fn=input_fn_predict)

In [91]:
predictions = list(pred_gen)
predictions
#for pred in estimator.predict(input_fn=input_fn_predict):
#    predctions.append(pred['predictions'])

INFO:tensorflow:Restoring parameters from /tmp/tmpji1u7zxn/model.ckpt-21000


[{'predictions': array([  2.79708352e+08], dtype=float32)},
 {'predictions': array([  1.57711584e+08], dtype=float32)},
 {'predictions': array([  2.52339040e+08], dtype=float32)},
 {'predictions': array([  2.79682560e+08], dtype=float32)},
 {'predictions': array([  4.98899232e+08], dtype=float32)},
 {'predictions': array([  6.63931264e+08], dtype=float32)},
 {'predictions': array([  3.77471584e+08], dtype=float32)},
 {'predictions': array([  3.49297312e+08], dtype=float32)},
 {'predictions': array([  3.56224640e+08], dtype=float32)},
 {'predictions': array([ 15080314.], dtype=float32)},
 {'predictions': array([  4.39185216e+08], dtype=float32)},
 {'predictions': array([  9.37571712e+08], dtype=float32)},
 {'predictions': array([  3.01035904e+08], dtype=float32)},
 {'predictions': array([  1.83943008e+08], dtype=float32)},
 {'predictions': array([  3.63461088e+08], dtype=float32)},
 {'predictions': array([  1.20848120e+08], dtype=float32)},
 {'predictions': array([  1.32283032e+08], dty

** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [49]:
from sklearn.metrics import mean_squared_error

In [92]:
#print(mean_squared_error(y_test,predctions))
final_preds = []

for pred in predictions:
    final_preds.append(pred['predictions'])

In [95]:
mean_squared_error(y_test,final_preds)

3.0179917278515149e+17

In [98]:
mean_squared_error(y_test,final_preds)**0.5

549362514.90718901

# Great Job!